# Digital Automatic Gain Control (AGC)
AGC is an algorithm used to maintain a received signal at a given amplitude or power level. The digital AGC transform simulates this effect, which introduces time-varing effects and transients into the signal based on the adaptation of the AGC.

In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import torchsig_complex_data_type

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = DatasetMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_max = 1,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

An example signal is created which is multiple QPSK bursts atop additive white gaussian noise (AWGN). The parameters are randomized to show different modes or effects that the AGC can apply, such as: increasing the gain, decreasing the gain, fast adaptation speeds, slow adaptation speeds, and others.

In [ ]:
# burst parameters
burst_length = 128
num_bursts = 4
N = burst_length*num_bursts*2

# define noise
receive_signal = np.sqrt(1e-6)*(np.random.normal(0,1,N) + 1j*np.random.normal(0,1,N))

# build bursts
time_index = np.arange(0,burst_length)
for burst_index in range(num_bursts):
    time_index += burst_length
    qpsk_data = generate_qpsk_signal(num_iq_samples = burst_length, scale = 1.0).data
    receive_signal[time_index] += qpsk_data
    time_index += burst_length


# determine min/max range for input in dB
receive_signal_db = np.log(np.abs(receive_signal))
receive_signal_max_db = np.max(receive_signal_db)
receive_signal_min_db = np.min(receive_signal_db)

# calculate ranges for how to set AGC reference level
ref_level_max_db = receive_signal_max_db+10
ref_level_min_db = receive_signal_min_db-10

# randomly select the reference level the AGC will set
ref_level_db = np.random.uniform(ref_level_min_db,ref_level_max_db)

# randomize initial gain
initial_gain_db = np.random.uniform(-3,3)

# randomize AGC params
alpha_smooth = 10**(np.random.uniform(-5,-3))
alpha_track = 10**(np.random.uniform(-4,-2))
alpha_acquire = 10**(np.random.uniform(-4,-3))
track_range_db = np.random.uniform(0.5,2)
alpha_overflow = 10**(np.random.uniform(-2,-1))

# define the operating bounds of the AGC
low_level_db = ref_level_min_db-10
high_level_db = ref_level_max_db+10

# simulate AGC
agc_out = F.digital_agc(
    data = receive_signal,
    initial_gain_db = initial_gain_db,
    alpha_smooth = alpha_smooth,
    alpha_track = alpha_track,
    alpha_overflow = alpha_overflow,
    alpha_acquire = alpha_acquire,
    ref_level_db = ref_level_db,
    track_range_db = track_range_db,
    low_level_db = low_level_db,
    high_level_db = high_level_db,
)

# calculate the actual gain applied
gain_profile = np.abs(agc_out) / np.abs(receive_signal)

fig = plt.figure(figsize=(12,8))
fig.subplots_adjust(hspace=0.5)

ax = fig.add_subplot(2,1,1)
ax.plot(10*np.log10(np.abs(receive_signal)),label='Input')
ax.plot(10*np.log10(np.abs(agc_out)),label='AGC Out')
ax.set_xlim([0,len(agc_out)])
ax.grid()
ax.set_title(f'Demonstration of AGC Effect')
ax.set_xlabel('Time Index $n$',fontsize='large')
ax.set_ylabel('Magnitude (dB)')
ax.legend(fontsize='large', loc='upper left');

ax = fig.add_subplot(2,1,2)
ax.plot(10*np.log10(gain_profile))
ax.set_xlim([0,len(gain_profile)])
ax.grid()
ax.set_title(f'AGC Gain')
ax.set_xlabel('Time Index $n$',fontsize='large')
ax.set_ylabel('AGC Gain (dB)')
